In [25]:
# imports needed and logging
import gzip
import gensim 
import logging
from gensim.models import doc2vec
import json
import os
import collections
import smart_open
import random

In [17]:
# test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
# lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
# lee_test_file = test_data_dir + os.sep + 'lee.cor'

lyric_train_file = "example_lyrics.json"
lyric_test_file = "example_test_lyrics.json"
train_data = {}
with open(lyric_train_file) as f:  
    train_data = json.load(f)
test_data = {}
with open(lyric_test_file) as f:  
    test_data = json.load(f)

# train_corpus = []
# count = 0
# for key,line in data.items():
#     train_corpus.append(gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [count]))
#     count+=1
# print(train_corpus[0])

In [14]:
def read_corpus(_data, tokens_only=False):
    i = 0
    for key,line in _data.items():
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])
        i+=1
            

In [18]:
train_corpus = list(read_corpus(train_data))
test_corpus = list(read_corpus(test_data, tokens_only=True))
print(train_corpus[:1])

[TaggedDocument(words=['yeah', 'this', 'album', 'is', 'dedicated', 'to', 'all', 'the', 'teachers', 'that', 'told', 'me', 'never', 'amount', 'to', 'nothin', 'to', 'all', 'the', 'people', 'that', 'lived', 'above', 'the', 'buildings', 'that', 'was', 'hustlin', 'in', 'front', 'of', 'called', 'the', 'police', 'on', 'me', 'when', 'was', 'just', 'tryin', 'to', 'make', 'some', 'money', 'to', 'feed', 'my', 'daughter', 'it', 'all', 'good', 'and', 'all', 'the', 'niggas', 'in', 'the', 'struggle', 'you', 'know', 'what', 'sayin', 'it', 'all', 'good', 'baby', 'baby', 'it', 'was', 'all', 'dream', 'used', 'to', 'read', 'word', 'up', 'magazine', 'salt', 'pepa', 'and', 'heavy', 'up', 'in', 'the', 'limousine', 'hangin', 'pictures', 'on', 'my', 'wall', 'every', 'saturday', 'rap', 'attack', 'mr', 'magic', 'marley', 'marl', 'let', 'my', 'tape', 'rock', 'til', 'my', 'tape', 'popped', 'smokin', 'weed', 'in', 'bambu', 'sippin', 'on', 'private', 'stock', 'way', 'back', 'when', 'had', 'the', 'red', 'and', 'black'

In [19]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [20]:
model.build_vocab(train_corpus)

In [21]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 161 ms, sys: 29.4 ms, total: 191 ms
Wall time: 188 ms


In [22]:
model.infer_vector(['we', 'will', 'rock', 'you'])

array([ 0.04687589, -0.01503481,  0.03382487, -0.00528468, -0.0259236 ,
       -0.01881934, -0.0241199 , -0.02302958,  0.05098238,  0.02495525,
        0.01949299,  0.02376367,  0.00277763, -0.03284064, -0.00383121,
       -0.00833727, -0.02387964, -0.00026465, -0.0551689 , -0.03285943,
        0.03448899, -0.01002029,  0.06430416, -0.02841065,  0.01611285,
       -0.02201502,  0.0144625 ,  0.03884128, -0.0165611 , -0.01525949,
       -0.00292809,  0.0333811 , -0.00985382, -0.03546102,  0.06238037,
       -0.01388395, -0.0001457 ,  0.04681368,  0.05883223, -0.00876173,
        0.0414249 ,  0.03254603,  0.04218918,  0.00704454, -0.06274661,
        0.02074599, -0.02484215,  0.05798219,  0.01415981, -0.00282075],
      dtype=float32)

In [26]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [27]:
collections.Counter(ranks)

Counter({0: 4})

In [28]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))


Document (3): «look was gonna go easy on you and not to hurt your feelings but only going to get this one chance something wrong can feel it six minutes slim shady you re on just feeling ve got like something about to happen but don know what if that means what think it means we re in trouble big trouble and if he is as bananas as you say not taking any chances you were just what the doctor ordered beginning to feel like rap god rap god all my people from the front to the back nod back nod now who thinks their arms are long enough to slap box slap box they said rap like robot so call me rapbot but for me to rap like computer must be in my genes got laptop in my back pocket my pen ll go off when half cock it got fat knot from that rap profit made living and killing off it ever since bill clinton was still in office with monica lewinsky feeling on his nut sack an mc still as honest but as rude and indecent as all hell syllables killaholic kill em all with this slickety gibbedy hibbedy hi

In [31]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (1): «yeah they wishin and wishin and wishin and wishin they wishin on me yuh been movin calm don start no trouble with me tryna keep it peaceful is struggle for me don pull up at am to cuddle with me you know how like it when you lovin on me don wanna die for them to miss me yes see the things that they wishin on me hope got some brothers that outlive me they gon tell the story shit was different with me god plan god plan hold back sometimes won yuh feel good sometimes don ay don finessed down weston road ay nessed might go down yeah wait go hard on southside yuh way make sure that north side eat and still bad things it lot of bad things that they wishin and wishin and wishin and wishin they wishin on me bad things it lot of bad things that they wishin and wishin and wishin and wishin they wishin on me yuh ay ay she say do you love me tell her only partly only love my bed and my momma sorry fifty dub even got it tatted on me they ll bring the crashers to the party and yo